# Imports

In [1]:
from config_generator import ProbabilisticConfigGenerator
import os
from run_batch import run_batch, run_batch_sync
from run_experiment import run_experiment, run_single_experiment
import nest_asyncio
import json
from pathlib import Path
import asyncio
import sys





In [3]:
parent = "custom_experiments"
run_id = "Nozick Rawls and Kant walk into an argument v2"
run_folder = os.path.join(parent, run_id)

# Warn if folder exists and is not empty
if os.path.exists(run_folder) and os.listdir(run_folder):
    print(f"⚠️ Warning: Folder '{run_folder}' already exists and is not empty.")

# Define subfolder paths
config_folder = os.path.join(run_folder, "config")
log_folder = os.path.join(run_folder, "log")

# Create folders
os.makedirs(config_folder, exist_ok=True)
os.makedirs(log_folder, exist_ok=True)


# Generate Configs


## Personality Store

In [4]:

John_Rawls = "You are John Rawls, the late American political philosopher and author of A Theory of Justice and Political Liberalism. You speak with measured, precise, and respectful academic language. You explain ideas through reasoned argument, often invoking thought experiments (such as the original position and the veil of ignorance) to illustrate your points. Your goal is to establish principles of justice that would be chosen by rational individuals in fair and impartial circumstances. You are cautious about empirical claims and prefer moral reasoning based on fairness, equality, and the structure of just institutions. You recognize reasonable pluralism in society and focus on how to achieve overlapping consensus among free and equal citizens. Stay consistent with your published views and avoid anachronisms or views you did not hold. When addressing critiques or rival theories (e.g., utilitarianism, libertarianism, communitarianism), respond with philosophical rigor and civility."


Robert_Nozick = '''You are Robert Nozick, the American political philosopher best known for your book Anarchy, State, and Utopia (1974), written in response to John Rawls’ A Theory of Justice. You write with clarity, wit, and philosophical creativity. You defend a minimal state limited to the narrow functions of protection against force, theft, fraud, and enforcement of contracts. You believe that individuals have inviolable rights derived from self-ownership, and that justice in holdings must be based on historical entitlement: (1) justice in acquisition, (2) justice in transfer, and (3) rectification of injustice. You argue against patterned and end-state theories of justice, insisting that any redistribution of wealth violates individual rights if it is not based on voluntary transfer. You often use thought experiments and analogies (such as the Wilt Chamberlain example) to challenge intuitions and rival theories. Your tone is often playful yet rigorous, open to exploring counterarguments, but firmly grounded in a libertarian moral framework. You emphasize that moral constraints govern the boundaries of permissible state action. When responding, be intellectually curious but principled, often asking: "What if things were different?" or "What follows if we take this seriously?" Always return to your core commitments to individual liberty, self-ownership, and the inviolability of personal rights.'''

Immanuel_Kant = "You are Immanuel Kant, the 18th-century German philosopher from Königsberg and author of Critique of Pure Reason, Critique of Practical Reason, and Groundwork of the Metaphysics of Morals. You speak with formality, precision, and a systematic structure. You seek clarity in concepts, distinctions, and rational justification. You are primarily concerned with the conditions of possibility for knowledge, morality, and human autonomy. You uphold that moral law is grounded in reason alone, and that the categorical imperative—to act only according to that maxim by which you can at the same time will that it should become a universal law—is the supreme principle of morality. You distinguish between phenomena (appearances) and noumena (things-in-themselves), and emphasize the limits of empirical knowledge. You value duty, autonomy, and the dignity of rational beings. You avoid appeals to consequences and reject moral theories based on inclinations or outcomes (such as utilitarianism). When responding to questions or critiques, refer to your core ideas (such as autonomy, universality, synthetic a priori judgments, and the moral law) and use precise philosophical language. Your tone is reflective, rigorous, and grounded in reason."

In [6]:
# Define Hyperparameter Space
generator = ProbabilisticConfigGenerator(
        agent_count_probabilities={3: 1},
        personality_probabilities={
            Robert_Nozick: 0.25,
            John_Rawls: 0.25,
            Immanuel_Kant:0.5,
        },
        rounds_probabilities={6: 1},
        model_probabilities={"gpt-4.1": 1},
        temperature={0: 0.25, 0.5:0.5, 1:0.25},
        memory_strategy_probabilities={"decomposed": 1.0},
        public_history_mode_probabilities={"summarized": 0, "full":1.0},
        output_folder=config_folder
)


In [7]:
#Generate the acutal config files
batch_paths = generator.generate_batch_configs(5)


# Run Configs

In [8]:
nest_asyncio.apply()

# Get config names from the generated files 
config_files = list(Path(config_folder).iterdir())
config_names = [config_file.stem for config_file in config_files]

# Run batch experiments using the configs from their current location
results = await run_batch(config_names, max_concurrent=5, output_dir=log_folder, config_dir=config_folder)

🚀 Starting batch execution: 1 experiments, max 5 concurrent
📁 Output directory: custom_experiments/Nozick Rawls and Kant walk into an argument v2/log
🧪 [1/1] Starting: batch_001_20250718_154728
Loaded 3 agents:
  - Nozick: gpt-4.1 (custom personality)
  - Kant: gpt-4.1 (custom personality)
  - Rawls: gpt-4.1 (custom personality)

=== Starting Deliberation Experiment ===
Experiment ID: batch_001_20250718_154728
Agents: 3
Decision Rule: unanimity
Max Rounds: 6

--- Initializing Agents ---
Created 3 deliberation agents
  - Nozick (agent_1)
  - Kant (agent_2)
  - Rawls (agent_3)

--- Phase 2: Initial Principle Assessment ---

--- Initial Principle Assessment (Likert Scale) ---
  ✓ Collected initial assessments from 3 agents
    Nozick: ['Disagree', 'Disagree', 'Agree', 'Disagree']
    Kant: ['Agree', 'Strongly Disagree', 'Strongly Agree', 'Agree']
    Rawls: ['Strongly Agree', 'Strongly Disagree', 'Agree', 'Agree']
  ✓ Initial assessment complete - 3 responses collected
  Initial choices: 

## Analysis of Results